Paper: https://www.nature.com/articles/s41467-023-41421-4
Data: https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-41421-4/MediaObjects/41467_2023_41421_MOESM5_ESM.xlsx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS
from sklearn.mixture import BayesianGaussianMixture
from joblib import Parallel, delayed
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [ ]:
# Read the Excel file into a pandas DataFrame and perform clean-up
df = pd.read_csv('asv_data.csv', skiprows=3, low_memory = False).iloc[:, 1:]
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
#Split df into 2 separate dfs for easier use 
metadata = df.iloc[:, 0:12]
metadata.columns = ["Sample ID", "Mother/Child", "Sex", "Age", "Dyad ID", "16S accession", 
                     "WMS accession", "Antibiotic exposure time", "Antibiotic", 
                     "Social Disadvantage score", "Psychological stress score", "Mother's IL-6 T3"]
metadata = metadata.iloc[3:, :]
metadata = metadata.set_index("Sample ID")

asv_16s = df.iloc[:, 2747:5820].rename(columns=df.iloc[0]).iloc[3:, :]
asv_16s = asv_16s.set_index(metadata.index) 

In [ ]:
#Clean data and select child data only for NMDS/DMM plot  
asv_16s_meta = pd.concat([metadata, asv_16s], axis=1)

#select rows that contain data for children only 
child_df = asv_16s_meta[asv_16s_meta['Mother/Child'] == 'Child']#.iloc[:,12:]
NMDS_data = child_df.iloc[:,12:]

#select columns that have at least 3 reads 
child_df = child_df.apply(pd.to_numeric, errors='coerce')
non_zero_counts = child_df.apply(lambda x: (x.astype(bool)).sum(), axis=0)
filtered_child_meta = child_df.loc[:, non_zero_counts >= 3] #create filtered df with metadata 
filtered_child_df = filtered_child_meta.iloc[:,12:] #filtered df without metadata 

In [ ]:
#Dimensionality Reduction 
def calculate_nmds(data):
    #calculate dissimilarity matrix 
    dis_matrix = pairwise_distances(data, metric='braycurtis')
    #perform 500 initialization of NMDS 
    mds = MDS(n_components=2, dissimilarity='precomputed', normalized_stress='auto', max_iter=500, n_init=500)
    mds_result = mds.fit_transform(dis_matrix)
    return mds_result

#DMM Clustering 
def cluster_iteration(data):
    #create Bayseiangaussianmixture model 
    bgm = BayesianGaussianMixture(n_components=2, random_state=42, max_iter=500, n_init=500,weight_concentration_prior_type='dirichlet_distribution')
    #assign clusters based on model 
    cluster_labels = bgm.fit_predict(data)
    #save clustering results to dataframe 
    df_meta = pd.DataFrame({'NMDS 1': mds_result[:, 0], 'NMDS 2': mds_result[:, 1], 'Cluster': cluster_labels})
    df_meta['Cluster'] = df_meta['Cluster'].map({0: 1, 1: 2})
    return df_meta

# NMDS calculation
mds_result = calculate_nmds(filtered_child_df)

best_df_meta = cluster_iteration(filtered_child_df)


In [ ]:
#Plot clustering results 

#set plot style and colors 
sns.set(style="white") 
colors = ["#00AFF0", "#FCC101"]

#Plot clustering results such that position of points are based on NMDS and color is based on clustering result 
sns.scatterplot(x='NMDS 1', y='NMDS 2', hue='Cluster', palette=colors, data=best_df_meta)
plt.legend(title='Cluster', loc='upper center', bbox_to_anchor=(1.2, 0.5), borderaxespad=0.)
plt.show()

In [ ]:
#extract cluster column
cluster_df = best_df_meta[['Cluster']]

#reset indices for both dataframes
filtered_child_meta_reset = filtered_child_meta.reset_index(drop=True)
cluster_df_reset = cluster_df.reset_index(drop=True)
#merge the two dataframes on their new indices
result_df = pd.concat([filtered_child_meta_reset, cluster_df_reset], axis=1)

In [ ]:
#set Seaborn style and colors
sns.set(style="white") 
colors = ["#D3EEFF", "#FFF2D2"]
palette = sns.set_palette(sns.color_palette(colors))

dot_colors = ["#00AFF0", "#FCC101"]

#create two subplots that share a y axis 
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

#plot social disadvantage score boxplots 
sns.boxplot(x='Cluster', y='Social Disadvantage score', data=result_df, ax=axes[0], palette=palette)

#plot individual datapoints on top of box plot 
sns.stripplot(x='Cluster', y='Social Disadvantage score', data=result_df, hue='Cluster', palette=dot_colors,
              edgecolor='black', linewidth=1, alpha=0.5, size=5, ax=axes[0])
axes[0].set_title('Social Disadvantage Score')
axes[0].get_legend().remove()
axes[0].set_ylim(bottom=result_df['Social Disadvantage score'].min() - 0.5,
                 top=result_df['Psychological stress score'].max() + 1.5)

#plot psychological stress score boxplots 
sns.boxplot(x='Cluster', y='Psychological stress score', data=result_df, ax=axes[1], palette=palette)
# plot individual datapoints on top of box plot
sns.stripplot(x='Cluster', y='Psychological stress score', data=result_df, hue='Cluster', palette=dot_colors,
              edgecolor='black',linewidth=1, alpha=0.5, size=5, ax=axes[1])
axes[1].set_title('Psychological Stress Score')

#merge plots to look closer to the original figures plots 
axes[1].set(ylabel='') 
axes[1].tick_params(left=False)  

#set y-axis label 
axes[0].set_ylabel('SD or PS score', fontsize=12)

# set style for axes 
for ax in axes:
    ax.tick_params(axis='both', labelsize=10)
    ax.set_xlabel('Cluster', fontsize=12)

#calculate p-values
for i, column in enumerate(['Social Disadvantage score', 'Psychological stress score']):
    data_cluster_1 = result_df[result_df['Cluster'] == 1][column]
    data_cluster_2 = result_df[result_df['Cluster'] == 2][column]

    #Mann-Whitney U-test
    stat, p_value = mannwhitneyu(data_cluster_1, data_cluster_2, alternative='two-sided')

    #FDR adjustment
    reject, p_value_corrected, _, _ = multipletests(p_value, method='fdr_bh')

    #add bracket annotation above the boxplots and display p value 
    x1, x2 = 0, 1  # columns 'Cluster 1' and 'Cluster 2'
    y, h, col = result_df[column].max() + 0.25, 0.5, 'k'

    axes[i].plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    axes[i].text((x1 + x2) * 0.5, y + h + 0.1, f'p = {p_value_corrected[0]:.2e}', ha='center', va='bottom', color=col, fontsize=10)

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0,hspace = 0) 

#set legend position 
handles, labels = axes[1].get_legend_handles_labels()
legend = axes[1].legend(handles, labels, title='Cluster', loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()
